# Kubernetes Service Deployment

This notebook showcases how a data owner could set up the service on a kubernetes cluster, add and make their data available to certain user. In addition, it also shows how to set up user sessions.

We use helm charts to deploy the service on a kubernetes cluster.

### Building the container images
The Lomas service is comprised of a fastapi server and a MongoDB database for keeping state about users and datasets. While the database image is public, the server image must first be built and pushed to a registry. This also holds true for the client session image.

As a preparation step, first make sure to have a docker registry at your disposal and log into that:

`docker login` (=> use personal token from dockerhub, has to be done only once)

This must be done only once, the local docker credential helper will store the token locally.

Let's now build the server image and push it to the registry. For this, we need to move from the root of this repository to `server/` and run `docker build`:

In [ ]:
!cd .. && docker build --target lomas_server -t <your_registry>/lomas_server:latest .
!cd .. && docker push <your_registry>/lomas_server:latest

This will copy the server code as well as some dummy datasets into the server image.

As a second step, let's do the same for the client image:

In [ ]:
!cd ../../client/ && docker build --target lomas_client -t <your_registry>/lomas_client:latest .
!cd ../../client/ && docker push <your_registry>/lomas_client:latest

### Starting the service

We use a Helm chart to deploy the service on a Kubernetes cluster. For the next part of this notebook to work correctly, one must have access to a Kubernetes cluster with sufficient rights and an environment with correctly configured helm and kubectl command line tools.

The lomas-server chart is located at `server/deploy/helm/charts/lomas_server`, let us change our working directory to this location.

In [90]:
import os
os.chdir('../deploy/helm/charts/lomas_server')

The `values.yaml` file contains all the configuration values for the service. We must now update the `image.repository` field to the one we pushed the server container image to. One can also change the url to which the service will be published with `ingress.hosts[0].host` and `ingress.tls.hosts[0]` (or disable this feature by setting `ingress.enabled` to `False`).

    => Update `values.yaml` file

Password and secrets will be deployed in kubernetes secrets and other service parameters will be deployed in configMaps.

As previously stated, the service is made up of a server and a MongoDB database. Before installing the chart, we must thus first download the MongoDB dependency.

In [91]:
!helm dependency update

Saving 1 charts
Pulled: registry-1.docker.io/bitnamicharts/mongodb:13.18.1
Digest: sha256:f3b2a691537260044746bc4a8898e9ae68e8c29864639737b6da920f99aebe97
Deleting outdated charts


Now the chart is ready to be installed, so let the magic happen!

In [92]:
!helm install -f values.yaml lomas-service .

W0918 07:49:26.628394  768420 warnings.go:70] annotation "kubernetes.io/ingress.class" is deprecated, please use 'spec.ingressClassName' instead
NAME: lomas-service
LAST DEPLOYED: Mon Sep 18 07:49:24 2023
NAMESPACE: user-paulineml
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
1. Get the application URL by running these commands:
  https://sdd-demo.lab.sspcloud.fr/


The installation notes show the url at which the server is exposed. One can have a look at the server state by checking `<server_url>/state` and the api docummentation by visiting `<server_url>/docs` in their browser.

One can also check the whether the service started without issues by using the `kubectl get all` command as well as inspecting the server logs with `kubectl logs <server-pod-name>`

### Starting the client session

The client deployment Helm chart is located in another directory, so let's again move our working directory

In [ ]:
os.chdir("../../../../../client/deploy/helm/charts/lomas_client")

Again, one needs to update the `values.yaml` file with the desired values. The important fields are `ingress.hosts[0].host` and `ingress.tls.hosts[0]` for the url, `password` for the user session and `image.repository` for specifying the previously built image. Make sure to change the `nameOverride`, `fullnameOverride` and url when deploying multiple client images.

`=> update values.yaml`

Similarly to the server deployment, let's install the client Helm chart:

In [ ]:
!helm install -f values.yaml lomas-client .

The user session should now be available at the specified url, just type the password and you are in!

If the service was started with developper mode to false, move to the `admin_notebook` to learn how to administer your freshly deployed service.

Once users and datasets have been added to the service, one can start to experiment with it. The URL to use for the service is the one defined in the server `values.yaml` file. 

### Stopping the service

To tear down the service, we simply execute the command `helm uninstall lomas-service`. The same goes for the client session with `helm uninstall lomas-client`.